In [10]:
!pip install nemo_toolkit['all']
!pip install hydra-core==1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 9.1 MB/s 
     |████████████████████████████████| 952 kB 48.9 MB/s 
     |████████████████████████████████| 86 kB 4.1 MB/s 
     |████████████████████████████████| 109 kB 52.2 MB/s 
     |████████████████████████████████| 235 kB 54.0 MB/s 
     |████████████████████████████████| 13.1 MB 48.4 MB/s 
     |████████████████████████████████| 99 kB 8.5 MB/s 
     |████████████████████████████████| 11.2 MB 49.2 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
     |████████████████████████████████| 1.3 MB 58.6 MB/s 
     |████████████████████████████████| 1.2 MB 18.5 MB/s 
     |████████████████████████████████| 68 kB 6.2 MB/s 
     |████████████████████████████████| 97 kB 6.7 MB/s 
     |████████████████████████████████| 769 kB 61.6 MB/s 
     |████████████████████████████████| 1.8 MB 58.9 MB/s 
     |████████████████████████████████| 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 144 kB 7.6 MB/s 
  Attempting uninstall: hydra-core
    Found existing installation: hydra-core 1.1.2
    Uninstalling hydra-core-1.1.2:
      Successfully uninstalled hydra-core-1.1.2


In [11]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
import omegaconf
from omegaconf import OmegaConf
from omegaconf import DictConfig
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.plugins import DDPPlugin
import sys
import logging

[NeMo W 2022-06-18 15:05:00 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.


In [ ]:
sys.argv = ['']
del sys

In [ ]:
LANGUAGE = "slices"
path = 'drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet'
manifest_dir = os.path.join(path, LANGUAGE)
train_manifest = f"{manifest_dir}/train/train.json"
dev_manifest = f"{manifest_dir}/dev/dev.json"
test_manifest = f"{manifest_dir}/test/test.json"

In [ ]:
@hydra_runner(config_path=r"drive/MyDrive/Colab Notebooks/Sprinklr_Project1/conformer", config_name="conformer_ctc_bpe")
def func(cfg):
  cfg['model']['train_ds']['manifest_filepath'] = train_manifest
  cfg['model']['validation_ds']['manifest_filepath'] = dev_manifest
  cfg['model']['test_ds']['manifest_filepath'] = test_manifest
  print(cfg['model']['validation_ds']['manifest_filepath'])
  checkpoint_callback = ModelCheckpoint(dirpath='drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet/', 
                                        save_last=True, save_top_k=20,
                                        filename='{epoch}-{val_wer:.2f}-{other_metric:.2f}',
                                        monitor="val_wer", every_n_epochs=10)
  checkpoint_path = None
  trainer = pl.Trainer(num_processes=1, max_epochs=1, callbacks=[checkpoint_callback], accelerator='cpu')
  asr_model = EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_large")
  asr_model._wer.log_prediction = True
  asr_model.set_trainer(trainer)
  param_config = DictConfig(cfg['model'])
  asr_model.setup_training_data(param_config.train_ds)
  asr_model.setup_multiple_validation_data(val_data_config=param_config.validation_ds)
  asr_model.setup_multiple_test_data(test_data_config=param_config.test_ds)
  asr_model.spec_augmentation = asr_model.from_config_dict(asr_model.cfg.spec_augment)
  asr_model.setup_optimization(DictConfig(cfg['model']['optim']))
  asr_model.encoder.unfreeze()
  asr_model.decoder.unfreeze()

  trainer.fit(asr_model, ckpt_path=checkpoint_path)
  checkpoint_callback.best_model_path
  checkpoint_callback.best_model_score
  trainer.save_checkpoint
   
  asr_model.save_to("drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet/result.nemo")
  if hasattr(cfg.model, 'test_ds') and cfg.model.test_ds.manifest_filepath is not None and False:
      gpu = 1 if cfg.trainer.gpus != 0 else 0
      test_trainer = pl.Trainer(
          gpus=gpu,
          precision=trainer.precision,
          amp_level=trainer.accelerator_connector.amp_level,
          amp_backend=cfg.trainer.get("amp_backend", "native"),
      )
      if asr_model.prepare_test(test_trainer):
        test_trainer.test(asr_model, verbose=True)

In [ ]:
func()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet/slices/dev/dev.json
[NeMo I 2022-06-18 12:46:10 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.9.0/stt_en_conformer_ctc_large/010120d9959425c7862c9843960b3235/stt_en_conformer_ctc_large.nemo.
[NeMo I 2022-06-18 12:46:10 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.9.0/stt_en_conformer_ctc_large/010120d9959425c7862c9843960b3235/stt_en_conformer_ctc_large.nemo
[NeMo I 2022-06-18 12:46:10 common:789] Instantiating model from pre-trained checkpoint
[NeMo I 2022-06-18 12:46:21 mixins:168] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2022-06-18 12:46:21 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-06-18 12:46:21 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2022-06-18 12:46:21 features:200] PADDING: 0


[NeMo W 2022-06-18 12:46:22 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                    not been set for this class (WER). The property determines if `update` by
                    default needs access to the full metric state. If this is not the case, significant speedups can be
                    achieved and we recommend setting this to `False`.
                    We provide an checking function
                    `from torchmetrics.utilities import check_forward_no_full_state`
                    that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                    default for now) or if `full_state_update=False` can be used safely.
                    
      warnings.warn(*args, **kwargs)
    
[NeMo W 2022-06-18 12:46:23 nemo_logging:349] /usr/local/lib/python3.7/dist-packa

...

[NeMo I 2022-06-18 12:46:33 wer_bpe:212] 
    
[NeMo I 2022-06-18 12:46:33 wer_bpe:213] reference:three
[NeMo I 2022-06-18 12:46:33 wer_bpe:214] predicted:three


[NeMo W 2022-06-18 12:46:33 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1931: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
      category=PossibleUserWarning,
    


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

[NeMo I 2022-06-18 12:47:02 wer_bpe:212] 
    
[NeMo I 2022-06-18 12:47:02 wer_bpe:213] reference:three
[NeMo I 2022-06-18 12:47:02 wer_bpe:214] predicted:free
[]


In [14]:
%cd /content/

/content


In [15]:
import numpy as np
asr_model = EncDecCTCModelBPE.restore_from("drive/MyDrive/Copy of result.nemo")

[NeMo I 2022-06-18 15:06:10 mixins:168] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2022-06-18 15:06:10 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet/slices/train/train.json
    sample_rate: 16000
    batch_size: 4
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    
[NeMo W 2022-06-18 15:06:10 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet/slices/dev/dev.json
    sample_rate: 16000
    batch_size: 4
    shuffle: false
    num_workers

[NeMo I 2022-06-18 15:06:10 features:200] PADDING: 0


[NeMo W 2022-06-18 15:06:12 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                    not been set for this class (WER). The property determines if `update` by
                    default needs access to the full metric state. If this is not the case, significant speedups can be
                    achieved and we recommend setting this to `False`.
                    We provide an checking function
                    `from torchmetrics.utilities import check_forward_no_full_state`
                    that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                    default for now) or if `full_state_update=False` can be used safely.
                    
      warnings.warn(*args, **kwargs)
    
[NeMo W 2022-06-18 15:06:12 nemo_logging:349] /usr/local/lib/python3.7/dist-packa

[NeMo I 2022-06-18 15:06:13 save_restore_connector:243] Model EncDecCTCModelBPE was successfully restored from /content/drive/MyDrive/Copy of result.nemo.


Creating a list of Paths

In [6]:
os.chdir('/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet/slices/test')
directory = 'recordings2'
manifest_dir_url = '/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet/slices/test/recordings2'
new_list_of_paths = []

for filename in os.listdir(directory):
    new_list_of_paths.append(f"{manifest_dir_url}/{filename}")
print(new_list_of_paths)

['/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet/slices/test/recordings2/2_jackson_34.wav', '/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet/slices/test/recordings2/0_theo_46.wav', '/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet/slices/test/recordings2/4_jackson_0.wav']


In [16]:
predictions = asr_model.transcribe(paths2audio_files=new_list_of_paths)
print(predictions)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2022-06-18 15:06:19 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    


['to', 'zero', 'four']


Creating list of true statements

In [17]:
os.chdir('/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet/slices/test')
directory = 'recordings2'
manifest_dir_url = '/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Tiny DataSet/slices/test/recordings2'
new_list_of_statements = []

m = {'0': "zero",
    '1': "one",
    '2': "two",
    '3': "three",
    '4': "four",
    '5': "five",
    '6': "six",
    '7': "seven",
    '8': "eight",
    '9': "nine"}

for filename in os.listdir(directory):
    new_list_of_statements.append(f"{m[filename[0]]}")
print(new_list_of_statements)

['two', 'zero', 'four']


In [19]:
!pip install evaluate
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 1.7 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149857 sha256=2740925adc97d6a449c537e885d6a5c5584189add27e4c9509322faa55c998d2
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [20]:
import evaluate
from evaluate import load
wer = load("wer")

WER using FineTuned Model

In [21]:
wer_score = wer.compute(predictions=predictions, references=new_list_of_statements)
print(wer_score)

0.3333333333333333


In [22]:
pretrained_model = EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_large")

[NeMo I 2022-06-18 15:11:21 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_large/versions/1.6.0/files/stt_en_conformer_ctc_large.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0/stt_en_conformer_ctc_large/010120d9959425c7862c9843960b3235/stt_en_conformer_ctc_large.nemo
[NeMo I 2022-06-18 15:11:33 common:789] Instantiating model from pre-trained checkpoint
[NeMo I 2022-06-18 15:11:38 mixins:168] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2022-06-18 15:11:38 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-06-18 15:11:38 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2022-06-18 15:11:38 features:200] PADDING: 0


[NeMo W 2022-06-18 15:11:40 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                    not been set for this class (WER). The property determines if `update` by
                    default needs access to the full metric state. If this is not the case, significant speedups can be
                    achieved and we recommend setting this to `False`.
                    We provide an checking function
                    `from torchmetrics.utilities import check_forward_no_full_state`
                    that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                    default for now) or if `full_state_update=False` can be used safely.
                    
      warnings.warn(*args, **kwargs)
    
[NeMo W 2022-06-18 15:11:40 nemo_logging:349] /usr/local/lib/python3.7/dist-packa

[NeMo I 2022-06-18 15:11:41 save_restore_connector:243] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0/stt_en_conformer_ctc_large/010120d9959425c7862c9843960b3235/stt_en_conformer_ctc_large.nemo.


WER using Pre-trained model

In [23]:
pretrained_model_predictions = pretrained_model.transcribe(paths2audio_files=new_list_of_paths)
wer_score = wer.compute(predictions=pretrained_model_predictions, references=new_list_of_statements)
print(wer_score)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2022-06-18 15:12:52 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    


0.3333333333333333
